In [4]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

/Users/sam/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/sam/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/sam/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/sam/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [9]:
# Open JSON file and assign to dict
f = open('/Users/sam/Downloads/DS 2021 Exercise/data_dict.json')
data_dict = json.load(f)
f.close()

In [10]:
for key in data_dict.keys():
    print(data_dict[key])

{'name': 'CurrentSpeed', 'units': 'knots', 'range': None, 'description': ' '}
{'name': 'CurrentDir', 'units': 'degrees', 'range': None, 'description': ' '}
{'name': 'TWS', 'units': 'knots', 'range': None, 'description': 'True Wind Speed'}
{'name': 'TWA', 'units': 'degrees', 'range': None, 'description': 'True Wind Angle'}
{'name': 'AWS', 'units': 'knots', 'range': None, 'description': 'Apparent Wind Speed'}
{'name': 'AWA', 'units': 'degrees', 'range': None, 'description': 'Apparent Wind Angle'}
{'name': 'Roll', 'units': 'degrees', 'range': None, 'description': 'Roll, also equals to -Heel'}
{'name': 'Pitch', 'units': 'degrees', 'range': None, 'description': 'Pitch angle'}
{'name': 'HeadingMag', 'units': 'degrees', 'range': None, 'description': 'magnetic heading'}
{'name': 'HoG', 'units': 'degrees', 'range': None, 'description': 'heading over ground'}
{'name': 'HeadingTrue', 'units': 'degrees', 'range': None, 'description': 'true heading. True heading - heading over ground = Yaw'}
{'name

In [5]:
# Import data and inspect top
data = pd.read_csv('/Users/sam/Downloads/test_data.csv')
data.head()

,CurrentSpeed,CurrentDir,TWS,TWA,AWS,AWA,Roll,Pitch,HeadingMag,HoG,...,VMG,RudderAng,Leeway,TWD,WSoG,VoltageDrawn,ModePilote,DateTime,Yaw,Tacking
0,0.0756,123.0,10.8,48.0,10.4,48.0,-3.54,9.080000,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.5,11.8,5.0,2019-04-14 00:00:00.000,-299.0,1.0
1,0.0756,123.0,10.8,48.0,10.4,48.0,-3.54,9.080000,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.5,11.8,5.0,2019-04-14 00:00:01.000,-299.0,1.0
2,0.0756,123.0,10.8,48.0,10.4,48.0,-3.52,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,9.9,11.8,5.0,2019-04-14 00:00:02.000,-299.0,1.0
3,0.0756,123.0,10.8,48.0,10.4,48.0,-3.52,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,9.9,11.8,5.0,2019-04-14 00:00:03.000,-299.0,1.0
4,0.0756,123.0,10.8,48.0,10.4,48.0,-3.50,9.099999,24.0,308.0,...,0.0594,4.666667,0.0,356.0,10.3,11.8,5.0,2019-04-14 00:00:04.000,-299.0,1.0


In [17]:
# Describe data
data.iloc[:,12:].describe()

,Longitude,Latitude,SoG,SoS,AvgSoS,VMG,RudderAng,Leeway,TWD,WSoG,VoltageDrawn,ModePilote,Yaw,Tacking
count,219836.000000,219840.000000,219842.000000,219840.000000,219838.000000,219837.000000,219838.000000,219839.000000,219838.000000,219836.000000,219839.000000,219839.000000,219834.000000,219995.000000
mean,-60.675999,16.805625,7.658572,7.607856,6.382535,4.109238,2.025093,-1.226548,83.323893,14.746686,12.417475,2.422614,5.595763,0.209273
std,0.982475,3.929849,3.075285,3.084592,2.903147,2.066419,4.963518,0.793367,53.810225,4.291340,0.570748,1.043669,140.521400,0.406791
min,-61.816873,11.971172,0.005400,0.000000,0.070200,0.000000,-37.333336,-10.000000,0.000000,0.000000,11.100000,2.000000,-359.000000,0.000000
25%,-61.639917,12.913855,7.684200,7.597800,5.637600,3.148200,-0.666667,-2.000000,61.000000,11.800000,12.100000,2.000000,-13.000000,0.000000
50%,-61.199546,15.232683,8.726400,8.699400,7.581600,4.579200,2.333333,-1.000000,70.000000,14.800000,12.300000,2.000000,-6.000000,0.000000
75%,-59.735390,20.902214,9.369000,9.298800,8.461801,5.583600,4.666667,-1.000000,82.000000,17.600000,12.500000,2.000000,-1.000000,0.000000
max,-59.279375,22.209945,12.598201,12.700800,8.532001,9.860400,47.000000,9.000000,359.000000,35.700001,14.200000,5.000000,359.000000,1.000000
